In [56]:
import numpy as np
import pandas as pd
from pycirclize import Circos

In [57]:
df = pd.read_excel('../data/br2f_topographic_data.xlsx', header=0, index_col=0)

In [58]:
# define left hemisphere
brain_regions = df.index.to_numpy()

left_hemisphere = brain_regions[:392]
df_left_hemisphere = df[left_hemisphere]
df_left_hemisphere = df_left_hemisphere.head(392)

## Hippocampal - Amygdala connectivity plots

Next we will look at three aggregated regions (CEA, MEA, BST) and two non-aggregated regions (AAA, IA) and their connectivity with the hippocampus. For the aggregated regions, we start by summing across the subregions.

In [78]:
# construct regions
CEA_regions = np.array(['CEAm','CEAl','CEAc'])
MEA_regions = np.array(['MEAav','MEAad','MEApv','MEApd'])
BST_regions = np.array([
    'BSTam','BSTfu','BSTv','BSTmg','BSTdm','BSTal','BSTov','BSTju','BSTrh','BSTpr','BSTif','BSTtr','BSTd','BSTse'
])

hippocampus_regions = np.array(['DG','CA3','CA2','CA1v','CA1d','PAR', 'SUBv'])

amygdala_non_aggregated_regions = np.array(['AAA','IA'])

amygdala_aggregated_regions = np.concatenate(
    (CEA_regions, MEA_regions, BST_regions), axis=0
)

amy_hipp_regions = np.concatenate(
    (amygdala_non_aggregated_regions, amygdala_aggregated_regions, hippocampus_regions), 
    axis=0
)

In [79]:
# reduce the left hemisphere dataframe to only include amygdala and hippocampal regions
df_amy_hipp = df_left_hemisphere[amy_hipp_regions]
df_amy_hipp = df_amy_hipp[df_amy_hipp.index.isin(amy_hipp_regions)]

In [80]:
#CEA
# rows
CEA_rows = df_amy_hipp[df_amy_hipp.index.isin(CEA_regions)].sum(axis=0)
df_amy_hipp.loc["CEA"] = CEA_rows

# delete extra LHA subregions from rows
df_amy_hipp = df_amy_hipp.loc[
~df_amy_hipp.index.isin(CEA_regions)
]

In [81]:
# columns
CEA_col_sum = df_amy_hipp[CEA_regions].sum(axis=1)
df_amy_hipp["CEA"] = CEA_col_sum

# delete extra LHA subregion from columns
df_amy_hipp = df_amy_hipp.drop(CEA_regions, axis=1)

In [82]:
# MEA
MEA_row_sum = df_amy_hipp[df_amy_hipp.index.isin(MEA_regions)].sum(axis=0)
df_amy_hipp.loc["MEA"] = MEA_row_sum

# delete extra DHA subregions from rows
df_amy_hipp = df_amy_hipp.loc[
~df_amy_hipp.index.isin(MEA_regions)
]

# columns
MEA_col_sum = df_amy_hipp[MEA_regions].sum(axis=1)
df_amy_hipp.loc[:,"MEA"] = MEA_col_sum

# delete extra LHA subregion from columns
df_amy_hipp = df_amy_hipp.drop(MEA_regions, axis=1)

In [83]:
# BST
BST_row_sum = df_amy_hipp[df_amy_hipp.index.isin(BST_regions)].sum(axis=0)
df_amy_hipp.loc["BST"] = BST_row_sum

# delete extra DHA subregions from rows
df_amy_hipp = df_amy_hipp.loc[
~df_amy_hipp.index.isin(BST_regions)
]

# columns
BST_col_sum = df_amy_hipp[BST_regions].sum(axis=1)
df_amy_hipp.loc[:,"BST"] = BST_col_sum

# delete extra LHA subregion from columns
df_amy_hipp = df_amy_hipp.drop(BST_regions, axis=1)

In [84]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST
Abbr.,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,6,0,31
IA,0,0,0,0,0,0,0,0,0,4,0,0
DG,0,0,0,6,0,0,0,0,0,0,0,0
CA3,0,0,3,0,6,6,6,0,3,0,0,0
CA2,0,0,4,4,0,4,0,2,3,0,0,0
CA1v,3,1,2,1,2,0,0,4,7,2,4,16
CA1d,0,0,2,0,0,0,0,1,0,0,0,0
SUBv,1,5,3,3,3,3,0,5,0,12,6,28
PAR,0,0,2,0,0,1,0,0,1,0,0,0


In [85]:
# remove self-connections (set diagonal to 0)
for i in range(len(df_amy_hipp.index)):
    df_amy_hipp.iloc[i,i] = 0

In [86]:
# we need to remove connections within the hypothalamus--
# we're only interested in hippocampal-hypothalamus connectivity

amygdala_regions = np.array(['AAA','IA', 'CEA','MEA','BST'])
for subregion_i in range(len(amygdala_regions)):
    for subregion_j in range(len(amygdala_regions)):
        df_amy_hipp.loc[
        amygdala_regions[subregion_i], 
        amygdala_regions[subregion_j]
        ] = 0

In [87]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST
Abbr.,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,0,0,0
IA,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,6,0,0,0,0,0,0,0,0
CA3,0,0,3,0,6,6,6,0,3,0,0,0
CA2,0,0,4,4,0,4,0,2,3,0,0,0
CA1v,3,1,2,1,2,0,0,4,7,2,4,16
CA1d,0,0,2,0,0,0,0,1,0,0,0,0
SUBv,1,5,3,3,3,3,0,0,0,12,6,28
PAR,0,0,2,0,0,1,0,0,0,0,0,0


In [88]:
#clear out intrinsic connections between hippocampus_regions
for subregion_i in range(len(hippocampus_regions)):
    for subregion_j in range(len(hippocampus_regions)):
        df_amy_hipp.loc[
        hippocampus_regions[subregion_i], 
        hippocampus_regions[subregion_j]
        ] = 0

In [89]:
df_amy_hipp

,AAA,IA,DG,CA3,CA2,CA1v,CA1d,PAR,SUBv,CEA,MEA,BST
Abbr.,,,,,,,,,,,,
AAA,0,0,0,0,0,4,0,0,0,0,0,0
IA,0,0,0,0,0,0,0,0,0,0,0,0
DG,0,0,0,0,0,0,0,0,0,0,0,0
CA3,0,0,0,0,0,0,0,0,0,0,0,0
CA2,0,0,0,0,0,0,0,0,0,0,0,0
CA1v,3,1,0,0,0,0,0,0,0,2,4,16
CA1d,0,0,0,0,0,0,0,0,0,0,0,0
SUBv,1,5,0,0,0,0,0,0,0,12,6,28
PAR,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
# remove rows of all zero
df_boop = df_amy_hipp.loc[:,(df_amy_hipp!=0).any()]

In [91]:
(df_boop.T!=0).any()

Abbr.
AAA      True
IA      False
DG      False
CA3     False
CA2     False
CA1v     True
CA1d    False
SUBv     True
PAR     False
CEA     False
MEA      True
BST      True
dtype: bool

In [92]:
df_boop = df_boop.loc[(df_boop.T!=0).any(),:]

In [93]:
df_boop

,AAA,IA,CA1v,PAR,SUBv,CEA,MEA,BST
Abbr.,,,,,,,,
AAA,0,0,4,0,0,0,0,0
CA1v,3,1,0,0,0,2,4,16
SUBv,1,5,0,0,0,12,6,28
MEA,0,0,5,3,6,0,0,0
BST,0,0,8,0,3,0,0,0


In [94]:
circos_amy_hipp_extrinsic_only = Circos.initialize_from_matrix(df_boop, cmap="tab10")

circos_amy_hipp_extrinsic_only.savefig("../output/amygdala_hypothalamus_extrinsic_connectivity.svg")